# Recurrent Neural Net From Scratch

I will skip the reasoning behind why we need RNN. There's plenty on the internet. I'll focus on learning the technical side, and hope that the answers to "why"s will come naturally. This is the same principal as other notebooks in my "from scratch" series. This is heavily based on the basic neural net, so read that one first.

Recall that in a neural network, we have:

![nn](imgs/nn.png)

For recurrent neural network, we carry over info from the previous data point in the sequence, by incorporating the values of the hidden layer, with its own weight $u$.

![nn](imgs/recurrent_neuralnet.png)

## Feed forward

Rephrasing the diagram, we have the feed forward process as, for each time step $t$,
 * $s_{h,t} = x_t W + a_{h,t-1}U + b_h$
 * $a_{h,t} = \sigma_h(s_{h,t})$
 * $s_{o,t} = a_{h,t}V +b_o$
 * $y_t = \sigma_o(s_{o,t})$

h means hidden layer, o means output layer, s means weighted sum, a means activated value. $\sigma$ means the activation function. For the activation functions, we use softmax at the output layer and tanh at the hidden layer.

$$\large
tanh(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}
$$

It has the convenient property that

$$ \large
(tanh(x))' = 1-(tanh(x))^2 
$$

## Backpropagation Through Time
* As there's dependency bewteen the previous and next steps, backpropagation in RNN goes through the entire sequence, called Backpropagation Through Time (BPTT) where we propagate the gradient not only through layers, but also through the entire sequence of data. This means we sum up the lost of the predictions of the sequece.

Overall loss is

$$
L_{total} = \sum_1^t L_t
$$

where

$$
L_t = -y_t log(\hat{y_t})
$$

* Backpropagation from the output layer to the hidden layer is similar as vanila neural network since we don't add anything new between these two layers
* Backpropagation from the hidden layer will go to both the input layer and the hidden layer of the previous data point.

### Weights from the output layer to the hidden layer

From the math of basic neural net, we know

$$ \large
\frac{\partial L_t}{\partial V} = (\hat{y_t} - y_t) \cdot a_{h,t}
$$

Replace $a_h$ with 1 for the bias term

### Weights from the hidden to the input layer and previous data point

From above setup, we can see that $s_{h,t}$ is an important term as it's a function of $W, U, a_{h,t-1}$. In backpropagation, the gradient will flow from $s_{h,t}$ to those components like the following, based on the formula in the feedforward section above:

$$\large
\begin{align}
\frac{\partial L_t}{\partial W} &= \frac{\partial L_t}{\partial s_{h,t}} \cdot x_t \\
\frac{\partial L_t}{\partial U} &= \frac{\partial L_t}{\partial s_{h,t}} \cdot a_{h,t-1} \\
\frac{\partial L_t}{\partial b_h} &= \frac{\partial L_t}{\partial s_{h,t}} 
\end{align}
$$

To get $\Large \frac{\partial L_t}{\partial s_{h,t}}$, we have

$$\large
\begin{align}
\frac{\partial L_t}{\partial s_{h,t}} & = \frac{\partial L_t}{\partial a_{h,t}}\cdot \frac{\partial a_{h,t}}{\partial s_{h,t}} \\
& = \frac{\partial L_t}{\partial a_{h,t}}\cdot \frac{\partial \sigma_h(s_{h,t})}{\partial s_{h,t}} \\
& = \frac{\partial L_t}{\partial a_{h,t}}\cdot (1-(tanh(s_{h,t}))^2) 
\end{align}
$$

All we have left to derive is $\Large \frac{\partial L_t}{\partial a_{h,t}}$. As $\large a_{h,t}$ contributes to $L_{t+1}$, the gradient from $L_{t+1}$ also goes to $\large a_{h,t}$. Therefore

$$\large
\begin{align}
\frac{\partial L_t}{\partial a_{h,t}} &= \frac{\partial L_t}{\partial s_{o,t}} \cdot \frac{\partial s_{o,t}}{\partial a_{h,t}} + \frac{\partial L_{t+1}}{\partial s_{h,t+1}} \cdot \frac{\partial s_{h,t+1}}{\partial a_{h,t}} \\
&= (\hat{y_t} - y_t)\cdot V + \frac{\partial L_{t+1}}{\partial s_{h,t+1}} \cdot U
\end{align}
$$

## Problem to solve

We'll use RNN to solve a toy problem of predicting the next number in mod 3. 1 is next to 0, 2 is next to 1, 0 is next to 2.

In [8]:
import numpy as np

# length of the sequence is the length of the problem. Keep it not too big for calculation, but not too small to be boring.
length = 50

x_num = [np.random.randint(0, 3) for i in range(length)]
y_num = [(x+1) % 3 for x in x_num]
x_str = ''.join([str(i) for i in x_num])
y_str = ''.join([str(i) for i in y_num])
print('input: ', x_str)
print('output:', y_str)

input:  10110101011110211012000212200102101112002100120200
output: 21221212122221022120111020011210212220110211201011


Repeat to make a dataset

In [148]:
def num_to_label(y, vocab=3):
    # turn a single digit label to multiclass labels
    labels = np.zeros((len(y), vocab), dtype=int)
    for i, yy in enumerate(y):
        labels[i, yy] = 1
    return labels

def make_pairs(length=10, vocab=3):
    x_num = [np.random.randint(0, vocab) for i in range(length)]
    y_num = [(x+1) % vocab for x in x_num]
    # One hot encode the input and output
    x = num_to_label(x_num)
    y = num_to_label(y_num)
    return x, y

In [149]:
size = 10
x, y = [], []
for i in range(size):
    xx, yy = make_pairs()
    x.append(xx)
    y.append(yy)

Different from basic neural net, RNN does feed forward and backpropagation for the entire sequence, not at each step independently. First let's layout the components we need. This should give a rough breakdown to help writing the code.

In [14]:
class RNN():
    def __init__(self):
        # initialize a random set of weigths and biases
        pass

    def predict(self):
        # this is the feed forward process
        pass

    def backpropagation(self):
        # one round of backpropagation through the entire sequence
        pass

    def train(self, epochs):
        pass

    def total_loss(self):
        pass

    def loss(self, t):
        return 

    def tanh(x):
        pass

    def softmax(x):
        pass

Then fill in the blanks. We'll first enable feed forward functions. In the init function, we not only need to initialize the parameters, but also reserve vectors to memorize all the calculations, as we'll rely on them in both feed forward and BTT. As we see above, terms like $\Large \frac{\partial L_t}{\partial s_{h,t}}$ are used multiple times and in different epochs.

In [159]:
import numpy as np

class RNN1():
    def __init__(self, n_nodes=10, vocab_size=3):
        # initialize a random set of weigths and biases
        self.n_nodes = n_nodes
        self.w = np.random.randn(n_nodes, vocab_size)
        self.u = np.random.randn(n_nodes, n_nodes)
        self.v = np.random.randn(vocab_size,n_nodes)
        self.b_h = np.random.randn(n_nodes)
        self.b_o = np.random.randn(vocab_size)
        self.s_h = []
        self.a_h = []
        self.s_o = []
        self.y_hat = []

    def predict(self, x):
        return [self.predict_single(xx) for xx in x]

    def predict_scores(self, x):
        return [self.feedforward(xx) for xx in x]
    
    def predict_single(self, x):
        scores = self.feedforward(x)
        return self.score_to_num(scores)

    def reset_mem(self):
        self.s_h = []
        self.a_h = []
        self.s_o = []
        self.y_hat = []
    
    def feedforward(self, x):
        self.reset_mem()
        # this is the feed forward process
        for t in range(len(x)):
            # this follows straight from the formula
            a_ht_prev = np.zeros(self.n_nodes) if t == 0 else self.a_h[t-1]
            s_ht = x[t] @ self.w.T + a_ht_prev @ self.u + self.b_h
            a_ht = self.tanh(s_ht)
            s_ot = a_ht @ self.v.T + self.b_o
            y_t = self.softmax(s_ot)

            # memorize all above
            self.s_h.append(s_ht)
            self.a_h.append(a_ht)
            self.s_o.append(s_ot)
            self.y_hat.append(y_t)
        return self.y_hat

    def total_loss(self, y, y_hat):
        return sum([self.loss(t, y, y_hat) for t in range(len(y))])

    @staticmethod
    def loss(t, y, y_hat):
        if (y[t] == y_hat[t]).all():
            return 0
        return np.sum(-y[t]*np.log(y_hat[t]))

    @staticmethod
    def tanh(x):
        return np.tanh(x)

    @staticmethod
    def softmax(x):
        x = np.array(x, dtype=np.float128)
        # to avoid overflow due to super large x, we subtract the max
        xx = x - np.max(x)
        exp = np.exp(xx)
        return exp / exp.sum(keepdims=True)

    @staticmethod
    def score_to_num(scores):
        indices = [np.argmax(p) for p in scores]
        return np.array(indices)

Run some random numbers to make sure matrix dimensions are good. We have written functions to
 * Make predictinos by feed forward, using randomly generated parameters
 * Calculate loss

In [160]:
rnn1 = RNN1()
pred_scores = rnn1.predict_scores(x)
pred = rnn1.predict(x)
loss = rnn1.total_loss(y, pred_scores)

In [162]:
pred, loss

([array([2, 0, 0, 0, 1, 2, 0, 2, 0, 0]),
  array([0, 0, 2, 0, 2, 1, 0, 0, 1, 2]),
  array([0, 0, 2, 0, 0, 0, 1, 0, 0, 2]),
  array([0, 0, 2, 0, 2, 0, 0, 0, 0, 0]),
  array([0, 0, 2, 0, 2, 2, 0, 0, 1, 0]),
  array([2, 0, 0, 0, 1, 2, 0, 2, 0, 2]),
  array([0, 0, 2, 2, 0, 0, 2, 1, 0, 0]),
  array([0, 0, 2, 0, 2, 2, 0, 2, 2, 0]),
  array([0, 0, 0, 1, 2, 0, 0, 2, 1, 0]),
  array([0, 0, 0, 0, 2, 0, 2, 0, 0, 1])],
 np.longdouble('206.90670911142617447'))

In [165]:
loss0 = rnn1.total_loss(y[0], y[0])
loss1 = rnn1.total_loss(pred_scores[0], pred_scores[0])
assert loss0 == 0
assert loss1 == 0

Move ahead to BPTT!

In [242]:
class RNN2(RNN1):
    def __init__(self, learning_rate=0.0001, n_nodes=10, vocab_size=3):
        super().__init__()
        self.learning_rate = learning_rate
        self.delta_l_over_s_h = []

    def backpropagation(self, x, y, y_hat):
        # BPTT for a single sequence of sample. This function
        # returns the deltas of W, V, U, b_h, and b_o
        T = len(y)
        du = np.zeros(self.u.shape)
        dv = np.zeros(self.v.shape)
        dw = np.zeros(self.w.shape)
        dbh = np.zeros(self.b_h.shape)
        dbo = np.zeros(self.b_o.shape)
        # btt goes backwards, from time t to time 0
        for t in reversed(range(T)):
            # one round of backpropagation at time t
            # output layer to hidden layer.
            # dLt/dV = (y_hat_t - y_t) * a_ht
            # dLt/dbo = (y_hat_t - y_t)
            test = self.a_h[t][:, np.newaxis]
            dbo_t = (y_hat[t] - y[t].T)
            dv_t = dbo_t[:, np.newaxis] @ test.T
            dv += dv_t
            dbo += dbo_t

            # hidden layer to input layer
            da_ht = (y_hat[)
            ds_ht = da_ht @ (1-self.a_hh[t] ** 2)
            dw_t = ds_ht @ x[t]
            
        return dw, dv, du, dbh, dbo

    def train(self, epochs, x, y, verbose=False):
        for i in range(epochs):
            for j in range(len(x)):
                # feed forward
                y_hat = self.feedforward(x[j])
                # calculate deltas
                dW, dV, dU, dbh, dbo = self.backpropagation(x[j], y[j], y_hat)
                # update weights accordingly
                self.u -= self.learning_rate * dU
                self.w -= self.learning_rate * dW
                self.v -= self.learning_rate * dV
                self.b_h -= self.learning_rate * dbh
                self.b_o -= self.learning_rate * dbo

In [243]:
rnn2 = RNN2()
rnn2.train(1, x, y)

In [204]:
a = np.array([1,2,3])[:, np.newaxis]
a.T, a.shape, a.T.shape

(array([[1, 2, 3]]), (3, 1), (1, 3))

TypeError: list indices must be integers or slices, not tuple